In [ ]:
# | default_exp _components.api_key

Note: 

While writing doc strings, please use the below syntax for linking methods/classes. So that the methods/classes gets highlighted in the browser and clicking on it will take the user to the linked function

    - To link a method from the class same file please use the `method_name` format.
    - To link a method from a different Class (can in a seperate file also) please use `Classname.method_name` format.

In [ ]:
from airt._testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import os
from datetime import date, datetime, timedelta

import pandas as pd
from fastcore.foundation import patch

from airt._components.client import Client
from airt._components.user import User
from airt._helper import (
    check_and_append_otp_query_param,
    delete_data,
    generate_df,
    get_attributes_from_instances,
    get_data,
    post_data,
)
from airt._logger import get_logger, set_level

In [ ]:
import logging
import os
import random
import string

import pytest

import airt._sanitizer
from airt._constant import SERVICE_PASSWORD, SERVICE_USERNAME
from airt._docstring.helpers import run_examples_from_docstring
from airt.client import User

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.INFO

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

20

[INFO] __main__: This is an info
[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
RANDOM_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
def mask(s: str) -> str:
    return "*" * len(s)


assert mask("test") == "****"

In [ ]:
# | export


class APIKey:
    """A class for managing the APIKeys in the server.

    Both the APIKey and the token can be used for accessing the airt services. However, there is a slight difference in generating and managing the two.

    For generating the APIKey, you first need to get the developer token. Please refer to `Client.get_token` method documentation to generate one.

    After logging in with your developer token, you can create any number of new APIKeys and can set an expiration date individually. You can also access
    other methods available in the APIKey class to list, revoke the APIKey at any time.

    Here's an example of how to use the APIKey class to create a new key and use it to access the airt service.

    Example:
        ```python
        # Importing necessary libraries
        from  airt.client import Client, APIKey, User

        # Authenticate
        Client.get_token(username="{fill in username}", password="{fill in password}")

        # Create a new key with the given name
        key_name = "{fill in key_name}"
        new_key = APIKey.create(name=key_name)

        # Display the details of the newly created key
        print(APIKey.details(apikey=key_name))

        # Call the set_token method to set the newly generated key
        Client.set_token(token=new_key["access_token"])

        # Print the logged-in user details
        # If set_token fails, the line below will throw an error.
        print(User.details())
        ```
    """

    API_KEY_COLS = ["uuid", "name", "created", "expiry", "disabled"]

    def __init__(
        self,
        uuid: str,
        name: Optional[str] = None,
        expiry: Optional[str] = None,
        disabled: Optional[bool] = None,
        created: Optional[str] = None,
    ):
        """Constructs a new APIKey instance.

        Args:
            uuid: APIKey uuid.
            name: APIKey name.
            expiry: APIKey expiry date.
            disabled: Flag to indicate the status of the APIKey.
            created: APIKey creation date.
        """
        self.uuid = uuid
        self.name = name
        self.expiry = expiry
        self.disabled = disabled
        self.created = created

    @staticmethod
    def create(
        name: str,
        expiry: Optional[Union[int, timedelta, datetime]] = None,
        otp: Optional[str] = None,
    ) -> Dict[str, str]:
        """Create a new APIKey

        In order to access the airt service with the newly generated APIKey, please call the `Client.set_token` method
        or set the APIKey value in the **AIRT_SERVICE_TOKEN** environment variable.

        !!! note

            - The APIKey's name must be unique. If not, an exception will be raised while creating a new key with the name of an existing key.
            However, you can create a new key with the name of a revoked key.

            - The expiry for an APIKey is optional, if not passed then the default value **None** will be used to create an APIKey with no expiry date!

        Args:
            name: The name of the APIKey.
            expiry: The validity for the APIKey. This can be an integer representing the number of days till expiry, can be
                an instance of timedelta (timedelta(days=x)) or can be an instance of datetime. If not passed, then the default value
                **None** will be used to create a APIKey that will never expire!
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if the MFA is enabled for your account.

        Returns:
            The APIKey and its type as a dictionary.

        Raises:
            ValueError: If the user is not authenticated.
            ValueError: If the user tries to create a new APIKey with an existing key name.
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        In the following example, a new APIKey is created with a 10-day expiration date and used to access the airt service.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Create a key with the given name and set the expiry to 10 days from now.
            # If the expiry parameter is not specified, a key with no expiry date is created.
            key_name = "{fill in key_name}"
            new_key_details = APIKey.create(name=key_name, expiry=10)

            # Display the details of the newly created key
            print(APIKey.details(apikey=key_name))

            # If a new key with the same name is created, an exception will be raised.
            # However, you can create a new key with the name of a revoked key.
            try:
                APIKey.create(name=key_name, expiry=10)
                print("Should not print this, the above line should raise an exception")
                raise RuntimeException()

            except ValueError as e:
                print("Expected to fail, everything is fine")

            # Finally, either call the below method to set the newly generated key
            # or store it in the AIRT_SERVICE_TOKEN environment variable.
            Client.set_token(token=new_key_details["access_token"])

            # If set_token fails, the line below will throw an error.
            print(APIKey.details(apikey=key_name))
            ```
        """
        if expiry is None:
            expiry_date = expiry
        else:
            if isinstance(expiry, int):
                delta = datetime.now() + timedelta(days=expiry)
            elif isinstance(expiry, timedelta):
                delta = datetime.now() + expiry
            else:
                delta = expiry

            expiry_date = delta.strftime("%Y-%m-%dT%H:%M")

        return Client._post_data(
            relative_url="/apikey",
            json=dict(name=name, expiry=expiry_date, otp=otp),
        )

    @staticmethod
    def as_df(ax: List["APIKey"]) -> pd.DataFrame:
        """Return the details of APIKey instances in a pandas dataframe.

        Args:
            ax: List of APIKey instances.

        Returns:
            Details of all the APIKeys in a dataframe.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.

        An example of displaying the APIKeys generated by the currently logged-in user in a dataframe

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Create a key without an expiry date in the given name
            key_name = "{fill in key_name}"
            APIKey.create(name=key_name)

            # Display all the APIKey instance details in a pandas dataframe
            df = APIKey.as_df(APIKey.ls())
            print(df)
            ```
        """
        lists = get_attributes_from_instances(ax, APIKey.API_KEY_COLS)  # type: ignore
        return generate_df(lists, APIKey.API_KEY_COLS)

    @staticmethod
    def ls(
        user: Optional[str] = None,
        offset: int = 0,
        limit: int = 100,
        include_disabled: bool = False,
    ) -> List["APIKey"]:
        """Return the list of APIKeys instances.

        Please do not pass the **user** parameter unless you are a super user. Only a super user can view
        the APIKeys created by other users.

        Args:
            user: user_uuid/username associated with the APIKey. Please call `User.details` method of the User class to get your user_uuid.
                If not passed, then the currently logged-in user_uuid will be used.
            offset: The number of APIKeys to offset at the beginning. If None, then the default value 0 will be used.
            limit: The maximum number of APIKeys to return from the server. If None, then the default value 100 will be used.
            include_disabled: If set to **True**, then the disabled APIKeys will also be included in the result.

        Returns:
            A list of APIKey instances.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.
            ValueError: If the user_uuid is invalid.

        An example of displaying the APIKeys generated by the currently logged-in user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Create a key without an expiry date in the given name
            key_name = "{fill in key_name}"
            APIKey.create(name=key_name)

            # Get the list of all APIKey instances created by the currently logged-in user.
            # If you are a super user, you can view the APIkeys created by other users by
            # passing their uuid/username in the user parameter.
            ax = APIKey.ls()
            print(ax)

            # Display the details of the instances in a pandas dataframe
            df = APIKey.as_df(ax)
            print(df)
            ```
        """
        user_uuid = User.details(user=user)["uuid"]

        apikeys = Client._get_data(
            relative_url=f"/{user_uuid}/apikey?include_disabled={include_disabled}&offset={offset}&limit={limit}"
        )

        ax = [
            APIKey(
                uuid=apikey["uuid"],
                name=apikey["name"],
                expiry=apikey["expiry"],
                disabled=apikey["disabled"],
                created=apikey["created"],
            )
            for apikey in apikeys
        ]

        return ax

    @staticmethod
    def details(apikey: str) -> pd.DataFrame:
        """Return details of an APIKey.

        Args:
            apikey: APIKey uuid/name.

        Returns:
            A pandas Dataframe encapsulating the details of the APIKey.

        Raises:
            ValueError: If the APIKey uuid is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        An example to get details of an APIKey

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Create a key without an expiry date in the given name
            key_name = "{fill in key_name}"
            APIKey.create(name=key_name)

            # Display the details of the newly created key
            print(APIKey.details(apikey=key_name))

            # To display the details of all keys created by the user, use the method below.
            df = APIKey.as_df(APIKey.ls())
            print(df)
            ```
        """
        details = Client._get_data(relative_url=f"/apikey/{apikey}")

        return pd.DataFrame(details, index=[0])[APIKey.API_KEY_COLS]

    @staticmethod
    def _get_key_names(keys: Union[List["APIKey"], List[str], str]) -> List[str]:
        """Get keys names

        Args:
            keys: Can be a string, list of string or a list of APIKey instances

        Returns:
            The APIKey names as a list
        """
        if isinstance(keys, str):
            return [keys]

        if all(isinstance(k, str) for k in keys):
            return keys  # type: ignore

        return [k.name for k in keys]  # type: ignore

    @staticmethod
    def revoke(
        keys: Union[str, List[str], List["APIKey"]],
        user: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> pd.DataFrame:
        """Revoke one or more APIKeys

        Please do not pass the **user** parameter unless you are a super user. Only a super user can revoke the
        APIKeys created by other users.

        Args:
            keys: APIKey uuid/name to revoke. To revoke multiple keys, either pass a list of APIKey uuid/names or a list of APIKey instances.
            user: user_uuid/username associated with the APIKey. Please call `User.details` method of the User class to get your user_uuid/username.
                If not passed, then the currently logged-in user will be used.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if the MFA is enabled for your account.

        Returns:
             A pandas Dataframe encapsulating the details of the deleted APIKey(s).

        Raises:
            ValueError: If the APIKey uuid is invalid.
            ValueError: If the user_uuid is invalid.
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        An example to revoke a single APIKey by name

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Create a key without an expiry date in the given name
            key_name = "{fill in key_name}"
            APIKey.create(name=key_name)

            # Check that the newly created key exists
            print([key.name for key in APIKey.ls()])

            # Revoke the newly created key
            # To delete multiple keys, pass a list of key names or key instances
            APIKey.revoke(keys=key_name)

            # Check that the newly created key does not exists
            print([key.name for key in APIKey.ls()])
            ```

        Here's an example of a super user revoking all APIkeys generated by a specific user.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, APIKey

            # Authenticate with super user privileges
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # List the APIKeys generated by a specific user
            user = "{fill in other_username}"
            ax = APIKey.ls(user=user)
            print([key.name for key in ax])

            # Revoke the APIKeys
            APIKey.revoke(keys=ax, user=user)

            # Check that all APIkeys have been revoked
            print([key.name for key in APIKey.ls(user=user)])
            ```
        """
        user_uuid = User.details(user=user)["uuid"]
        _keys = APIKey._get_key_names(keys)

        response_list = []

        for key_uuid in _keys:
            url = f"/{user_uuid}/apikey/{key_uuid}"
            response = Client._delete_data(
                relative_url=check_and_append_otp_query_param(url, otp)
            )
            response_list.append(response)

        return generate_df(response_list, APIKey.API_KEY_COLS)

In [ ]:
# tests for _get_key_names

_input = "key_name"
expected = "key_name"
actual = APIKey._get_key_names(_input)
display(actual)

_input = ["key_name_1", "key_name_2", "key_name_3"]
expected = ["key_name_1", "key_name_2", "key_name_3"]
actual = APIKey._get_key_names(_input)
display(actual)


class T:
    name = "key_name"


_input = [T(), T(), T()]
expected = ["key_name", "key_name", "key_name"]
actual = APIKey._get_key_names(_input)
display(actual)

['key_name']

['key_name_1', 'key_name_2', 'key_name_3']

['key_name', 'key_name', 'key_name']

In [ ]:
def generate_random_name(size=15, chars=string.ascii_uppercase + string.digits):
    return "".join(random.choice(chars) for _ in range(size))


assert len(generate_random_name()) == 15
assert type(generate_random_name()) == str

In [ ]:
# Run example for APIKey

username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]

run_examples_from_docstring(
    APIKey,
    username=username,
    password=password,
    key_name=generate_random_name(),
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey, User                                                               │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a new key with the given name                                                                      │
│     key_name = "{fill in key_name}"                                                                             │
│     new_key = APIKey.create(name=key_name)                                                                      │
│                                                                                                                 │
│     # Display the details of the newly created key                                                              │
│     print(APIKey.details(apikey=key_name))                                                                      │
│                                                                                                                 │
│     # Call the set_token method to set the newly generated key                                                  │
│     Client.set_token(token=new_key["access_token"])                                                             │
│                                                                                                                 │
│     # Print the logged-in user details                                                                          │
│     # If set_token fails, the line below will throw an error.                                                   │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                    uuid             name  ... expiry disabled                               │ │
│ │ 0  5fd74b8e-b4a0-4055-a2e2-1b8192733d63  I1LMY6MZAH7CGPS  ...   None    False                               │ │
│ │                                                                                                             │ │
│ │ [1 rows x 5 columns]                                                                                        │ │
│ │ {'username': 'johndoe', 'first_name': 'John', 'last_name': 'Doe', 'email': 'johndoe@airt.ai',               │ │
│ │ 'subscription_type': 'small', 'super_user': False, 'phone_number': None, 'uuid':                            │ │
│ │ '06a385d1-66a1-4ffc-8306-7f5821902fcc', 'disabled': False, 'created': '2022-11-02T08:17:09',                │ │
│ │ 'is_phone_number_verified': False, 'is_mfa_active': False}                                                  │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── st

In [ ]:
# Run example for APIKey.create

username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]

run_examples_from_docstring(
    APIKey.create,
    username=username,
    password=password,
    key_name=generate_random_name(),
)

<module>:13: No type or annotation for parameter 'name'
<module>:14: No type or annotation for parameter 'expiry'
<module>:17: No type or annotation for parameter 'otp'
<module>:21: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a key with the given name and set the expiry to 10 days from now.                                  │
│     # If the expiry parameter is not specified, a key with no expiry date is created.                           │
│     key_name = "{fill in key_name}"                                                                             │
│     new_key_details = APIKey.create(name=key_name, expiry=10)                                                   │
│                                                                                                                 │
│     # Display the details of the newly created key                                                              │
│     print(APIKey.details(apikey=key_name))                                                                      │
│                                                                                                                 │
│     # If a new key with the same name is created, an exception will be raised.                                  │
│     # However, you can create a new key with the name of a revoked key.                                         │
│     try:                                                                                                        │
│         APIKey.create(name=key_name, expiry=10)                                                                 │
│         print("Should not print this, the above line should raise an exception")                                │
│         raise RuntimeException()                                                                                │
│                                                                                                                 │
│     except ValueError as e:                                                                                     │
│         print("Expected to fail, everything is fine")                                                           │
│                                                                                                                 │
│     # Finally, either call the below method to set the newly generated key                                      │
│     # or store it in the AIRT_SERVICE_TOKEN environment variable.                                               │
│     Client.set_token(token=new_key_details["access_token"])                                                     │
│                                                                                                                 │
│     # If set_token fails, the line below will throw an error.                                                   │
│     print(APIKey.details(apikey=key_name))                                                                      │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── st

In [ ]:
# Run example for APIKey.as_df

run_examples_from_docstring(
    APIKey.as_df, key_name=generate_random_name(), username=username, password=password
)

<module>:3: No type or annotation for parameter 'ax'
<module>:6: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a key without an expiry date in the given name                                                     │
│     key_name = "{fill in key_name}"                                                                             │
│     APIKey.create(name=key_name)                                                                                │
│                                                                                                                 │
│     # Display all the APIKey instance details in a pandas dataframe                                             │
│     df = APIKey.as_df(APIKey.ls())                                                                              │
│     print(df)                                                                                                   │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                     uuid  ... disabled                                                      │ │
│ │ 0   37c644b3-4a18-4689-881e-30ed5b6bd1c7  ...    False                                                      │ │
│ │ 1   9c0d3637-15d9-4a8f-ab71-8fc188ab4492  ...    False                                                      │ │
│ │ 2   3d1a2643-440a-4d89-916f-63540b6e9edc  ...    False                                                      │ │
│ │ 3   5bfcbe9e-0dd8-4475-baf1-360228348353  ...    False                                                      │ │
│ │ 4   8b58a44a-1bb0-4d91-8753-be392bdbb613  ...    False                                                      │ │
│ │ 5   2a5759d3-2995-439c-9914-b55579aa9bd8  ...    False                                                      │ │
│ │ 6   5fe476d8-48c3-4464-b57d-bf4f1109710d  ...    False                                                      │ │
│ │ 7   b6d27167-38d5-492c-86aa-8920fafcba5c  ...    False                                                      │ │
│ │ 8   77207e77-5453-4055-a8bf-f3379ea4a1fb  ...    False                                                      │ │
│ │ 9   2a85864e-74d7-4392-8415-9703454ecce1  ...    False                                                      │ │
│ │ 10  4e577454-aa0a-4c1e-9fe8-240b03f5cb18  ...    False                                                      │ │
│ │ 11  7b25cd3b-ea98-4bdd-8336-48104536010d  ...    False                                                      │ │
│ │ 12  bd66c94f-0033-4506-8353-2785cc6623b2  ...    False                                                      │ │
│ │ 13  f45edd6a-9eb6-4854-b1a0-064078cd006f  ...    False                                                      │ │
│ │ 14  55263b34-fbd2-4ebb-87ee-e842acb4e8ed  ...    False                                                      │ │
│ │ 15  9305ae9b-8cd3-467d-8b5e-bb7a244ae720  ...    Fal

In [ ]:
# Run example for APIKey.ls

run_examples_from_docstring(
    APIKey.ls, key_name=generate_random_name(), username=username, password=password
)

<module>:6: No type or annotation for parameter 'user'
<module>:8: No type or annotation for parameter 'offset'
<module>:9: No type or annotation for parameter 'limit'
<module>:10: No type or annotation for parameter 'include_disabled'
<module>:13: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a key without an expiry date in the given name                                                     │
│     key_name = "{fill in key_name}"                                                                             │
│     APIKey.create(name=key_name)                                                                                │
│                                                                                                                 │
│     # Get the list of all APIKey instances created by the currently logged-in user.                             │
│     # If you are a super user, you can view the APIkeys created by other users by                               │
│     # passing their uuid/username in the user parameter.                                                        │
│     ax = APIKey.ls()                                                                                            │
│     print(ax)                                                                                                   │
│                                                                                                                 │
│     # Display the details of the instances in a pandas dataframe                                                │
│     df = APIKey.as_df(ax)                                                                                       │
│     print(df)                                                                                                   │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ [<airt.client.APIKey object>, <airt.client.APIKey object>,              │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>, <airt.client.APIKey object>,               │ │
│ │ <airt.client.APIKey object>]                                                              │ │
│ │                                     uuid  ... disabled                                                      │ │
│ │ 0   37c644b3-4a18-4689-881e-30ed5b6bd1c7  ...    False                                                      │ │
│ │ 1 

In [ ]:
# Run example for APIKey.details
run_examples_from_docstring(
    APIKey.details,
    key_name=generate_random_name(),
    username=username,
    password=password,
)

<module>:3: No type or annotation for parameter 'apikey'
<module>:6: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a key without an expiry date in the given name                                                     │
│     key_name = "{fill in key_name}"                                                                             │
│     APIKey.create(name=key_name)                                                                                │
│                                                                                                                 │
│     # Display the details of the newly created key                                                              │
│     print(APIKey.details(apikey=key_name))                                                                      │
│                                                                                                                 │
│     # To display the details of all keys created by the user, use the method below.                             │
│     df = APIKey.as_df(APIKey.ls())                                                                              │
│     print(df)                                                                                                   │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                    uuid             name  ... expiry disabled                               │ │
│ │ 0  941cb4ea-b453-4f52-808a-ce6a4d06755c  618R65M0PTOWIF6  ...   None    False                               │ │
│ │                                                                                                             │ │
│ │ [1 rows x 5 columns]                                                                                        │ │
│ │                                     uuid  ... disabled                                                      │ │
│ │ 0   37c644b3-4a18-4689-881e-30ed5b6bd1c7  ...    False                                                      │ │
│ │ 1   9c0d3637-15d9-4a8f-ab71-8fc188ab4492  ...    False                                                      │ │
│ │ 2   3d1a2643-440a-4d89-916f-63540b6e9edc  ...    False                                                      │ │
│ │ 3   5bfcbe9e-0dd8-4475-baf1-360228348353  ...    False                                                      │ │
│ │ 4   8b58a44a-1bb0-4d91-8753-be392bdbb613  ...    False                                                      │ │
│ │ 5   2a5759d3-2995-439c-9914-b55579aa9bd8  ...    False                                                      │ │
│ │ 6   5fe476d8-48c3-4464-b57d-bf4f1109710d  ...    False                                                      │ │
│ │ 7   b6d27167-38d5-492c-86aa-8920fafcba5c  ...    False                                                      │ │
│ │ 8   77207e77-5453-4055-a8bf-f3379ea4a1fb  ...    Fal

In [ ]:
# Run example for APIKey.revoke

# Create a new user and revoke all their apikeys
Client.get_token(username=os.environ["AIRT_SERVICE_SUPER_USER"])

random_name = generate_random_name()
_user_name = random_name
_password = random_name
_email = f"{random_name}@email.com"

response_df = User.create(
    username=_user_name,
    first_name="random_first_name",
    last_name="random_last_name",
    email=_email,
    password=_password,
    super_user=False,
    subscription_type="test",
)
assert response_df.shape == (1, len(User.USER_COLS))

run_examples_from_docstring(
    APIKey.revoke,
    username=username,
    password=password,
    key_name=generate_random_name(),
    super_user_username=os.environ["AIRT_SERVICE_SUPER_USER"],
    super_user_password=password,
    other_username=_user_name,
)
# loggin in back as default user
Client.get_token()

<module>:6: No type or annotation for parameter 'keys'
<module>:7: No type or annotation for parameter 'user'
<module>:9: No type or annotation for parameter 'otp'
<module>:13: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Create a key without an expiry date in the given name                                                     │
│     key_name = "{fill in key_name}"                                                                             │
│     APIKey.create(name=key_name)                                                                                │
│                                                                                                                 │
│     # Check that the newly created key exists                                                                   │
│     print()                                                                                                     │
│                                                                                                                 │
│     # Revoke the newly created key                                                                              │
│     # To delete multiple keys, pass a list of key names or key instances                                        │
│     APIKey.revoke(keys=key_name)                                                                                │
│                                                                                                                 │
│     # Check that the newly created key does not exists                                                          │
│     print()                                                                                                     │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ ['SRHE6D4DNOPZON6', 'YY96A5USMFKVLR2', 'Z8D61SQQIBBPTDA', 'D7WPXUSL7HSE39U', 'MLQAANQ3JG2P2LE',             │ │
│ │ 'HYEWVV6LGVYP8XE', '1XSBMNX2LS4W4ET', '14SMS5QRUJ78YPX', '47QN5TNTD9L773E', 'BK1ED7XSCYZD4DH',              │ │
│ │ 'YV7UAHEPG9KBJ05', 'N7LWCXWLDWBN89P', 'PHS18A5DNBYBIAF', 'XACPIZ1XC179M1L', 'QUUF17ZMWFJ9UQK',              │ │
│ │ 'PYQF1BXXM4R3L8K', 'MZP7KPGTWYY9E17', 'CXW83S6H53OTGJ3', '7BSAGI37I8MDIPJ', 'I1LMY6MZAH7CGPS',              │ │
│ │ 'IQ37Y5NEMKAIKPZ', 'ZN5H0ZE9AIVV3CQ', '2R85BB4WSKEBXD8', '618R65M0PTOWIF6', 'NR2ZYP83JT7M0EZ']              │ │
│ │ ['SRHE6D4DNOPZON6', 'YY96A5USMFKVLR2', 'Z8D61SQQIBBPTDA', 'D7WPXUSL7HSE39U', 'MLQAANQ3JG2P2LE',             │ │
│ │ 'HYEWVV6LGVYP8XE', '1XSBMNX2LS4W4ET', '14SMS5QRUJ78YPX', '47QN5TNTD9L773E', 'BK1ED7XSCYZD4DH',              │ │
│ │ 'YV7UAHEPG9KBJ05', 'N7LWCXWLDWBN89P', 'PHS18A5DNBYBIAF', 'XACPIZ1XC179M1L', 'QUUF17ZMWFJ9UQK',              │ │
│ │ 'PYQF1BXXM4R3L8K', 'MZP7KPGTWYY9E17', 'CXW83S6H53OTGJ3', '7BSAGI37I8MDIPJ', 'I1LMY6MZAH7CGPS',              │ │
│ │ 'IQ37Y5NEMKAIKPZ', 'ZN5H0ZE9AIVV3CQ', '2R85BB4WSKEBXD8', '618R65M0PTOWIF6']                                 │ │
│ │                                                     

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, APIKey                                                                     │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # List the APIKeys generated by a specific user                                                             │
│     user = "{fill in other_username}"                                                                           │
│     ax = APIKey.ls(user=user)                                                                                   │
│     print()                                                                                                     │
│                                                                                                                 │
│     # Revoke the APIKeys                                                                                        │
│     APIKey.revoke(keys=ax, user=user)                                                                           │
│                                                                                                                 │
│     # Check that all APIkeys have been revoked                                                                  │
│     print()                                                                                                     │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ []                                                                                                          │ │
│ │ []                                                                                                          │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Tests for APIKeys.create
# Positive scenario. Passing the expiry as int
Client.get_token()
res = APIKey.create(name=generate_random_name(), expiry=5)

display(f"{mask(res['access_token'])=}")
assert len(res["access_token"]) > 0, len(res["access_token"])

"mask(res['access_token'])='*************************************************************************************************************************************************************************************************'"

In [ ]:
# Tests for APIKeys.create
# Positive scenario. Passing the expiry as timedelta

res = APIKey.create(name=generate_random_name(), expiry=timedelta(days=5))

display(f"{mask(res['access_token'])=}")
assert len(res["access_token"]) > 0

"mask(res['access_token'])='*************************************************************************************************************************************************************************************************'"

In [ ]:
# Tests for APIKeys.create
# Positive scenario. Passing the expiry as datetime

res = APIKey.create(name=generate_random_name(), expiry=datetime(2022, 5, 17))

display(f"{mask(res['access_token'])=}")
assert len(res["access_token"]) > 0

"mask(res['access_token'])='*************************************************************************************************************************************************************************************************'"

In [ ]:
# Tests for APIKeys.create
# Positive scenario. Passing the expiry as datetime

res = APIKey.create(name=generate_random_name(), expiry=datetime(2022, 5, 17))

display(f"{mask(res['access_token'])=}")
assert len(res["access_token"]) > 0

"mask(res['access_token'])='*************************************************************************************************************************************************************************************************'"

In [ ]:
# Tests for APIKeys.create
# Positive scenario. Not passing expiry date

res = APIKey.create(name=generate_random_name())

display(f"{mask(res['access_token'])=}")
assert len(res["access_token"]) > 0, len(res["access_token"])

"mask(res['access_token'])='***************************************************************************************************************************************************************************'"

In [ ]:
# Tests for APIKeys.create
# Negative scenario. Passing existing key name

key_name = generate_random_name()
res = APIKey.create(name=key_name, expiry=datetime(2022, 5, 17))

with pytest.raises(ValueError) as e:
    APIKey.create(name=key_name, expiry=5)

display(str(e.value))
assert str(e.value) == "An Api-key with the same name already exists"

'An Api-key with the same name already exists'

In [ ]:
# Tests for APIKeys.create
# Negative scenario. Non-MFA user passing OTP parameter while creating new API key

with pytest.raises(ValueError) as e:
    random_otp = 123456
    APIKey.create(name=key_name, expiry=5, otp=random_otp)
display(str(e.value))

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# Tests for APIKey.ls

apikeys = APIKey.ls()
len(apikeys) > 0

assert isinstance(apikeys[0], APIKey)

# Testing list without offset and limit
df = APIKey.as_df(APIKey.ls())

display(df.uuid)

assert 0 < df.shape[0]

# Testing list with offset and limit
offset = 0
limit = 3

df = APIKey.as_df(APIKey.ls(offset=offset, limit=limit))

display(df.uuid)

assert 0 < df.shape[0] <= limit

# Testing list with invalid offset and limit
offset = 1_000_000_000
limit = 3

df = APIKey.as_df(APIKey.ls(offset=offset, limit=limit))
display(df.uuid)

assert df.shape == (0, 5)

0     37c644b3-4a18-4689-881e-30ed5b6bd1c7
1     9c0d3637-15d9-4a8f-ab71-8fc188ab4492
2     3d1a2643-440a-4d89-916f-63540b6e9edc
3     5bfcbe9e-0dd8-4475-baf1-360228348353
4     8b58a44a-1bb0-4d91-8753-be392bdbb613
5     2a5759d3-2995-439c-9914-b55579aa9bd8
6     5fe476d8-48c3-4464-b57d-bf4f1109710d
7     b6d27167-38d5-492c-86aa-8920fafcba5c
8     77207e77-5453-4055-a8bf-f3379ea4a1fb
9     2a85864e-74d7-4392-8415-9703454ecce1
10    4e577454-aa0a-4c1e-9fe8-240b03f5cb18
11    7b25cd3b-ea98-4bdd-8336-48104536010d
12    bd66c94f-0033-4506-8353-2785cc6623b2
13    f45edd6a-9eb6-4854-b1a0-064078cd006f
14    55263b34-fbd2-4ebb-87ee-e842acb4e8ed
15    9305ae9b-8cd3-467d-8b5e-bb7a244ae720
16    559391e6-20a7-4603-bd90-47506c5474d1
17    0c9b02cf-a890-43d0-8bb5-d730b92fdadf
18    a27e01d9-fe4a-464f-9a6f-4ec7d99059e5
19    5fd74b8e-b4a0-4055-a2e2-1b8192733d63
20    653905a5-a385-4510-b207-44868ef29fae
21    3877c87d-564b-43eb-b782-4fa726d4c72e
22    3be53309-2863-4502-a28e-14526041ece9
23    941cb

0    37c644b3-4a18-4689-881e-30ed5b6bd1c7
1    9c0d3637-15d9-4a8f-ab71-8fc188ab4492
2    3d1a2643-440a-4d89-916f-63540b6e9edc
Name: uuid, dtype: object

Series([], Name: uuid, dtype: float64)

In [ ]:
# Tests for APIKey.ls
# Negative scenario. Normal user accessing other's apikey

with pytest.raises(ValueError) as e:
    APIKey.ls(user=RANDOM_UUID_FOR_TESTING)

display(str(e.value))
assert str(e.value) == "Insufficient permission to access other user's data"

"Insufficient permission to access other user's data"

In [ ]:
# Tests for APIKey.ls
# Positive scenario. Super user accessing other's apikey

# logging in as normal user
Client.get_token()
expected = APIKey.as_df(APIKey.ls()).shape
normal_user_uuid = User.details()["uuid"]
normal_user_name = User.details()["username"]

# logging in as super user
Client.get_token(username=os.environ["AIRT_SERVICE_SUPER_USER"])
actual = APIKey.as_df(APIKey.ls(user=normal_user_uuid)).shape
assert actual == expected, f"{actual=}, {expected=}"
expected

actual = APIKey.as_df(APIKey.ls(user=normal_user_name)).shape
assert actual == expected, f"{actual=}, {expected=}"
expected

# Negative scenario. Super user accessing invalid user's apikey
with pytest.raises(ValueError) as e:
    APIKey.ls(user=RANDOM_UUID_FOR_TESTING)

display(str(e.value))

# login back as normal user
Client.get_token()

'The user uuid is incorrect. Please try again.'

In [ ]:
# Tests for APIKey.details
# Getting details by passing api_key name

APIKey.details(apikey=APIKey.ls()[0].name)

,uuid,name,created,expiry,disabled
0,37c644b3-4a18-4689-881e-30ed5b6bd1c7,SRHE6D4DNOPZON6,2022-11-02T08:46:49,None,False


In [ ]:
# Tests for APIKey.details
# Testing negative scenario. Passing invalid key

with pytest.raises(ValueError) as e:
    APIKey.details(apikey=RANDOM_UUID_FOR_TESTING)

display(e.value)

ValueError('No such apikey or not enough authorization to access the apikey.')

In [ ]:
# Tests for APIKey.revoke
# revoking by passing key_name and id

# creating sample keys
for i in range(5):
    APIKey.create(name=generate_random_name())

key_name = APIKey.ls()[0].name
key_uuid = APIKey.ls()[1].uuid
APIKey.revoke(keys=[key_name, key_uuid])

,uuid,name,created,expiry,disabled
0,37c644b3-4a18-4689-881e-30ed5b6bd1c7,SRHE6D4DNOPZON6,2022-11-02T08:46:49,None,True
1,9c0d3637-15d9-4a8f-ab71-8fc188ab4492,YY96A5USMFKVLR2,2022-11-02T08:46:49,2022-11-07T08:46:00,True


In [ ]:
# Tests for APIKey.revoke
# Passing list of api key ids
APIKey.revoke(keys=[APIKey.ls()[0].uuid, APIKey.ls()[1].uuid, APIKey.ls()[2].uuid])

,uuid,name,created,expiry,disabled
0,3d1a2643-440a-4d89-916f-63540b6e9edc,Z8D61SQQIBBPTDA,2022-11-02T08:46:50,2022-11-07T08:46:00,True
1,5bfcbe9e-0dd8-4475-baf1-360228348353,D7WPXUSL7HSE39U,2022-11-02T08:46:50,2022-05-17T00:00:00,True
2,8b58a44a-1bb0-4d91-8753-be392bdbb613,MLQAANQ3JG2P2LE,2022-11-02T08:46:50,2022-05-17T00:00:00,True


In [ ]:
# Tests for APIKey.revoke
# revoking by passing key_name and id

# cerating sample keys
for i in range(5):
    APIKey.create(name=generate_random_name())

key_name = APIKey.ls()[0].name
key_uuid = APIKey.ls()[1].uuid
APIKey.revoke(keys=[key_name, key_uuid])

,uuid,name,created,expiry,disabled
0,2a5759d3-2995-439c-9914-b55579aa9bd8,HYEWVV6LGVYP8XE,2022-11-02T08:46:50,None,True
1,5fe476d8-48c3-4464-b57d-bf4f1109710d,1XSBMNX2LS4W4ET,2022-11-02T08:46:50,2022-05-17T00:00:00,True


In [ ]:
# Tests for APIKey.revoke
# Testing negative scenario. Passing invalid key

with pytest.raises(ValueError) as e:
    APIKey.revoke(keys=RANDOM_UUID_FOR_TESTING)

display(e.value)

ValueError('No such apikey or not enough authorization to access the apikey.')

In [ ]:
# Tests for APIKey.revoke
# Testing negative scenario. Normal user deleting other user's key

with pytest.raises(ValueError) as e:
    random_user_id = RANDOM_UUID_FOR_TESTING
    random_key_id = RANDOM_UUID_FOR_TESTING
    APIKey.revoke(keys=random_key_id, user=random_user_id)

display(e.value)
assert str(e.value) == "Insufficient permission to access other user's data"

ValueError("Insufficient permission to access other user's data")

In [ ]:
# Tests for APIKey.revoke
# Positive scenario. Super user deleting other's apikey
Client.get_token()
normal_user_uuid = User.details()["uuid"]
expected_api_key_uuid = APIKey.ls()[0].uuid

display(f"{expected_api_key_uuid=}")

Client.get_token(username=os.environ["AIRT_SERVICE_SUPER_USER"])
actual_api_key_uuid = APIKey.revoke(
    keys=expected_api_key_uuid, user=normal_user_uuid
).uuid[0]
assert (
    actual_api_key_uuid == expected_api_key_uuid
), f"{actual_api_key_uuid=}, {expected_api_key_uuid=}"

# Negative scenario. Super user accessing invalid apikey
with pytest.raises(ValueError) as e:
    APIKey.revoke(keys=RANDOM_UUID_FOR_TESTING, user=normal_user_uuid)


# Negative scenario. Super user accessing invalid user
with pytest.raises(ValueError) as e:
    APIKey.revoke(keys=expected_api_key_uuid, user=RANDOM_UUID_FOR_TESTING)


# loggin back as normal user
Client.get_token()
display("OK")

"expected_api_key_uuid='b6d27167-38d5-492c-86aa-8920fafcba5c'"

'OK'

In [ ]:
# Tests for APIKey.revoke
# Testing negative scenario. Non-MFA user passing otp


APIKey.create(name=generate_random_name())
key_name = APIKey.ls()[0].name
with pytest.raises(ValueError) as e:
    random_otp = 123456
    APIKey.revoke(keys=key_name, otp=random_otp)

display(e.value)

ValueError('MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.')

In [ ]:
# Positive scenario: Super user revoking all keys created by a specific user

# login as super user
Client.get_token(username=os.environ["AIRT_SERVICE_SUPER_USER"])
# create a new user
random_name = generate_random_name()
_user_name = random_name
_password = random_name
_email = f"{random_name}@email.com"
response_df = User.create(
    username=_user_name,
    first_name="random_first_name",
    last_name="random_last_name",
    email=_email,
    password=_password,
    super_user=False,
    subscription_type="test",
)
display(response_df[["uuid", "username", "created"]])
assert response_df.shape == (1, len(User.USER_COLS))

# login as new user
Client.get_token(username=_user_name, password=_password)
# create new apikeys
for i in range(3):
    APIKey.create(name=generate_random_name())
ax = APIKey.ls()
display(APIKey.as_df(ax))
assert len(ax) == 3, len(ax)

# login as super user
Client.get_token(username=os.environ["AIRT_SERVICE_SUPER_USER"])
# List the APIKeys generated by a specific user
ax = APIKey.ls(user=_user_name)
display(APIKey.as_df(ax))
assert len(ax) == 3, len(ax)
# Revoke the APIKeys
APIKey.revoke(keys=ax, user=_user_name)
# Check that all APIkeys have been revoked
ax = APIKey.ls(user=_user_name)
display(APIKey.as_df(ax))
assert len(ax) == 0, len(ax)

# loggin back with default user credentials
Client.get_token()

,uuid,username,created
0,1469d528-6900-49a2-8c77-a64470e54f5c,IT5PYBZWFZINKFH,2022-11-02T08:49:34


,uuid,name,created,expiry,disabled
0,31b8a072-3baa-4113-b1f0-c4c3799f42c9,3AXQ1P9VGEF2H3K,2022-11-02T08:49:35,None,False
1,7439b44b-0ed7-44ac-b70e-27a81057b392,IGUWJDAMIPMS9CN,2022-11-02T08:49:35,None,False
2,4db4e416-76b6-4279-b61a-3222892c2019,0ATEMGYO7SS6PER,2022-11-02T08:49:35,None,False


,uuid,name,created,expiry,disabled
0,31b8a072-3baa-4113-b1f0-c4c3799f42c9,3AXQ1P9VGEF2H3K,2022-11-02T08:49:35,None,False
1,7439b44b-0ed7-44ac-b70e-27a81057b392,IGUWJDAMIPMS9CN,2022-11-02T08:49:35,None,False
2,4db4e416-76b6-4279-b61a-3222892c2019,0ATEMGYO7SS6PER,2022-11-02T08:49:35,None,False


,uuid,name,created,expiry,disabled
